# Sizing script for a rocket engine

## Introduction
This script is used to size the Eve engine.
The script is designed to be used in a Jupyter Notebook.

## Running the script
To run the script, simply run all cells in the notebook.

## Imports
Let's start by importing the necessary libraries.

In [88]:
from math import sqrt

## Constants

In [89]:
g0 = 9.80665  # m/s^2
R = 287.05  # J/kg/K
gamma = 1.4  # Cp/Cv

## Design parameters
The next cell contains the design parameters for the engine.
We start from a thrust target and a chamber pressure target.

In [90]:
# Main design parameters
thrust_target = 100  # N
chamber_pressure = 1e6  # Pa
I_sp = 260  # s

# Nozzle parameters
atm_pressure = 101325  # Pa
exit_pressure = atm_pressure  # Pa

# Propellant parameters
mixture_ratio = 2.327  # oxidizer/fuel
fuel_density = 800  # kg/m^3
oxidizer_density = 1141  # kg/m^3

## Propellant exit conditions

In [91]:
# Propellant exhaust conditions
exhaust_speed = I_sp * g0  # m/s
print(f"Exhaust speed: {exhaust_speed:.2f} m/s")

Exhaust speed: 2549.73 m/s


## Flow rates
The mass flow rate is calculated from the thrust and the chamber pressure.

In [92]:
mass_flow_rate = thrust_target / I_sp / g0
print(f"Mass flow rate: {mass_flow_rate:.3f} kg/s")

# Calculate oxidizer/fuel mass ratio
fuel_mass_flow_rate = mass_flow_rate / (mixture_ratio + 1)
oxidizer_mass_flow_rate = fuel_mass_flow_rate * mixture_ratio

print(f"Oxidizer mass flow rate: {oxidizer_mass_flow_rate:.3f} kg/s \t {oxidizer_mass_flow_rate*1000:.3f} g/s")
print(f"Fuel mass flow rate: {fuel_mass_flow_rate:.3f} kg/s \t {fuel_mass_flow_rate*1000:.3f} g/s")

# Calculate oxidizer/fuel volume ratio
oxidizer_volume_flow_rate = oxidizer_mass_flow_rate / oxidizer_density
fuel_volume_flow_rate = fuel_mass_flow_rate / fuel_density

print(f"Oxidizer volume flow rate: {oxidizer_volume_flow_rate:.6f} m^3/s \t {oxidizer_volume_flow_rate*1000:.3f} L/s")
print(f"Fuel volume flow rate: {fuel_volume_flow_rate:.6f} m^3/s \t {fuel_volume_flow_rate*1000:.3f} L/s")

Mass flow rate: 0.039 kg/s
Oxidizer mass flow rate: 0.027 kg/s 	 27.432 g/s
Fuel mass flow rate: 0.012 kg/s 	 11.788 g/s
Oxidizer volume flow rate: 0.000024 m^3/s 	 0.024 L/s
Fuel volume flow rate: 0.000015 m^3/s 	 0.015 L/s
